In [1]:
import pandas as pd
import re
import requests
import json

from bs4 import BeautifulSoup
from requests import get
from time import sleep
# from requests.adapters import HTTPAdapter
# from requests.packages.urllib3.util.ssl_ import create_urllib3_context

In [2]:
# decieve the agent // 브라우져 바꿔가면서 크롤링하게끔
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'} #user agent를 내 pc로 설정 

In [3]:
# load data
d = pd.read_csv('./base_data/powderroom_item_list.csv')

In [4]:
# stack data
concern={100:"모공",200:"각질",300:"주름",400:"잡티",500:"건조함",600:"피부톤",700:"민감성",800:"트러블",900:"아토피",1000:"피지과다"}
s_type={100:"건성",200:"중성",300:"지성",400:"복합성"}
item_id = []
product_name = []
age_s=[]
tags_s=[]
repurchased_s=[]
like_c_s=[]
concern_list_s=[]
type_s=[]
rate_s=[]
text_s=[]

In [5]:
# initialize
item_count = 0 

In [6]:
# crawl body
while True:
    if item_count not in d.item_id.index:
        break
    
    ####------------ 새로 들어온 리뷰 판단. -------------------------------
#     day_decision = [(datetime.fromtimestamp(time()) - datetime.fromtimestamp(int(str(rev['$created'])[:10]))).days for rev in reviews]
#     new_rev_len = len([i for i in day_decision if i == 0])
#     [:new_rev_len] 이거 추가
    ####-------------------------------------------------------------------
    
    url_p='https://www.powderroom.co.kr/api/products/'+ str(d.item_id[item_count])
    html_p = get(url_p, headers = headers).text
    products = json.loads(html_p)
    pg = int(products['count']['review']/20)+1
    schaft = ''
    
    for pn in range(pg):
        url='https://www.powderroom.co.kr/api/products/'+str(d.item_id[item_count])+'/reviews?order=new' + str(schaft)
        html = get(url, headers = headers).text
        reviews = json.loads(html)

        for i in range(20):
            try:
                try:
                    age_s.append(reviews[i]['user']['age']) # 나이
                    # item_id, name
                    item_id.append(d.item_id[item_count])
                    product_name.append(d.brand_name[item_count])       
                except KeyError:
                    item_id.append(d.item_id[item_count])
                    product_name.append(d.brand_name[item_count])  
                    age_s.append('')
                    
                try:
                    tags_s.append(reviews[i]['tags']) # tag
                except KeyError:
                    tag_s.append('')
                    
                try:
                    repurchased_s.append(reviews[i]['repurchased']) # 재구매 여부
                except KeyError:
                    repurchased_s.append('')
                    
                try:
                    like_c_s.append(reviews[i]['count']['like']) # 리뷰 좋아요 개수 
                except KeyError:
                    like_c_s.append(0)

                try:
                    concern_list=[]
                    for a in reviews[i]['user']['skin']['concerns']:
                        concern_list.append(concern[a])
                    concern_list_s.append(",".join(concern_list))
                except KeyError:
                    concern_list_s.append('')
                   
                try:
                    type_s.append(s_type[reviews[i]['user']['skin']['type']])
                except KeyError:
                    type_s.append('')
                    
                try:
                    rate_s.append(reviews[i]['rate']) # 평점
                except KeyError:
                    rate_s.append('')
            
                try:
                    text_s.append(reviews[i]['text']) # 리뷰
                except KeyError:
                    text_s.append('')
    
            except Exception:
                break
        
        # search after...
        search_after = [rev['_searchAfter'] for rev in reviews][-1]
    
    print(item_count)
    sleep(2)
    item_count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [9]:
product_information = pd.DataFrame(
    {'item_id': item_id,
     'product_name': product_name,
     'age':age_s,
     'tags':tags_s,
     'repurchased':repurchased_s,
     'like_c':like_c_s,
     'concern_list':concern_list_s,
     'type':type_s,
     'score':rate_s,
     'text':text_s
    })

In [10]:
product_information.to_csv('powderroom_2.csv',encoding='UTF-8',index_label=False)